# Skyline

In [1]:
import pandas as pd

url = 'https://elderscrolls.fandom.com/wiki/Bows_(Skyrim)'
tables = pd.read_html(url)
bows = tables[0]
bows


,Name,Unnamed: 1,Unnamed: 2,Unnamed: 3,Speed,Upgrade,Perk,Item ID
0,Ancient Nord Bow,8,12,45,0.8750,Steel Ingot,Steel,000302CA
1,Angi's Bow,7,7,50,0.9370,Steel Ingot,Steel,000CC392
2,Auriel's Bow DG,13,11,1000,1.0000,Refined Moonstone,Elven,xx000800
3,Bow of the Hunt,10,7,434,0.9370,Steel Ingot,Steel,000AB705
4,Daedric Bow,19,18,2500,0.5000,Ebony Ingot,Daedric,000139B5
5,Dragonbone Bow DG,20,20,2725,0.7500,Dragon Bone,Dragon,xx0176f1
6,Drainspell Bow,14,6,458,0.8750,NaN,None,000F82FC
7,Dravin's Bow,7,7,50,0.9370,Leather Strips,Steel,0006B9AD
8,Dwarven Bow,12,10,270,0.7500,Dwarven Metal Ingot,Dwarven,00013995
9,Dwarven Black Bow of Fate DR,13,10,1446,0.7500,Ebony Ingot,Dwarven,xx02c01a


In [2]:
bows = bows.iloc[:, 0:5]
bows.columns = ['name', 'damage', 'weight', 'gold', 'speed']
bows.head()


,name,damage,weight,gold,speed
0,Ancient Nord Bow,8,12,45,0.875
1,Angi's Bow,7,7,50,0.937
2,Auriel's Bow DG,13,11,1000,1.000
3,Bow of the Hunt,10,7,434,0.937
4,Daedric Bow,19,18,2500,0.500


In [3]:
bows.corr(numeric_only=True)


,damage,weight,gold,speed
damage,1.000000,0.613041,0.616495,-0.656327
weight,0.613041,1.000000,0.573820,-0.405975
gold,0.616495,0.573820,1.000000,-0.569465
speed,-0.656327,-0.405975,-0.569465,1.000000


Insights:

- The more damage, the heavier
- The more damage, the more expensive
- The more damage, the slower

One way to simplify is to look at the damage per second (DPS) instead of the damage per hit. This is a more accurate measure of the damage output of a weapon. In our dataset, the `speed` column indicates the number of arrows fired per second, and not the time between two shots. Therefore, the DPS is simply the product of the `damage` and `speed` columns.

In [8]:
bows['damage_per_second'] = bows['damage'] * bows['speed']
bows.sort_values(by='damage_per_second', ascending=False).head()


,name,damage,weight,gold,speed,damage_per_second
26,Karliah's Bow,25,9,5,0.625,15.625
5,Dragonbone Bow DG,20,20,2725,0.750,15.000
2,Auriel's Bow DG,13,11,1000,1.000,13.000
19,Gauldur Blackbow,14,18,530,0.875,12.250
20,Gauldur Blackbow,14,18,750,0.875,12.250


I'm not aware of a prepackaged way to calculate skylines, so we'll do it ourselves. The following implementation is naive: it iterates over the list of weapons and checks if the current weapon is dominated by any other weapon. It thus runs in $O(n^2)$ time, where $n$ is the number of weapons. There is a smarter algorithm based on block-nested loops that runs in $O(n \log n)$ time.

In [13]:
def a_dominates_b(a, b, to_min, to_max):

    n_better = 0

    for f in to_min:
        if a[f] > b[f]:
            return False
        n_better += a[f] < b[f]

    for f in to_max:
        if a[f] < b[f]:
            return False
        n_better += a[f] > b[f]

    if n_better > 0:
        return True
    return False


def find_skyline_brute_force(df, to_min, to_max):

    rows = df.to_dict(orient='index')
    skyline = set()

    for i in rows:
        dominated = False

        for j in rows:
            if i == j:
                continue

            if a_dominates_b(rows[j], rows[i], to_min, to_max):
                dominated = True
                break

        if not dominated:
            skyline.add(i)

    return df[df.index.isin(skyline)]

to_min = ['weight']
to_max = ['damage_per_second']

skyline = find_skyline_brute_force(
    bows,
    to_min=to_min,
    to_max=to_max
)
skyline


,name,damage,weight,gold,speed,damage_per_second
6,Drainspell Bow,14,6,458,0.875,12.250
16,Froki's Bow,6,5,307,1.000,6.000
26,Karliah's Bow,25,9,5,0.625,15.625
27,Long Bow,6,5,30,1.000,6.000


For a given bow that is part of the skyline, we can find all the bows that are worse than it.

In [26]:
all_worse = (
    (bows[to_min] > skyline.loc[6, to_min]).all(axis='columns') &
    (bows[to_max] < skyline.loc[6, to_max]).all(axis='columns')
)
bows[all_worse]


,name,damage,weight,gold,speed,damage_per_second
0,Ancient Nord Bow,8,12,45,0.8750,7.0000
1,Angi's Bow,7,7,50,0.9370,6.5590
3,Bow of the Hunt,10,7,434,0.9370,9.3700
4,Daedric Bow,19,18,2500,0.5000,9.5000
7,Dravin's Bow,7,7,50,0.9370,6.5590
8,Dwarven Bow,12,10,270,0.7500,9.0000
9,Dwarven Black Bow of Fate DR,13,10,1446,0.7500,9.7500
10,Ebony Bow,17,16,1440,0.5620,9.5540
11,Elven Bow,13,12,470,0.6870,8.9310
12,Falmer Bow,12,15,135,0.7500,9.0000


We can also visualize the skyline on a chart.

In [25]:
import altair as alt

(
    alt.Chart(
        bows
        .assign(is_skyline=lambda df: df.index.isin(skyline.index))
    )
    .mark_point()
    .encode(
        x='weight:Q',
        y='damage_per_second:Q',
        color='is_skyline:N',
        tooltip='name'
    )
    .interactive()
)


alt.Chart(...)

Application at Carbonfact: many different materials with different variants. It's good to be able to understand what compromises are possible. Furthermore, there's no point considering a material that is worse on all aspects. Therefore a skyline is a useful tool to sift out the materials that are not worth considering.